In [ ]:
import pandas as pd
import numpy as np
import os
from Functions import matching_check
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import joblib
# import graphviz

In [ ]:
# read files
FA = pd.read_csv('betsi_origin/FA.csv', sep=';' ,encoding='iso-8859-1', decimal=',')
FB_Klimat = pd.read_csv('betsi_origin/FB_Klimat.csv',sep=';' ,encoding='iso-8859-1', decimal=',').drop(columns=['Hustyp'])
year = pd.read_csv('betsi_origin/Byggår.csv', encoding='iso-8859-1')
FB49_YTTERVAGG = pd.read_csv('betsi_origin/FB49 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')
FB52_YTTERVAGG = pd.read_csv('betsi_origin/FB52 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')

In [ ]:
# concat several features
F = pd.concat([FA, FB_Klimat],axis=1, join='inner')

# Delete unrealistic values
F = F[(F['FB52 U-medelvärde ytterväggar']>=0.1) & (F['FB87-97 U-medelvärde fönster']>0)]
F.loc[:, 'FXByggår'] = year['FXByggÃ¥r']

# select muilti-family buildings
F = F[F['Hustyp']=='F']

In [ ]:
# building type
bt = F.loc[:,['F14ATempt', 'FA16AntalPlanOvanMark', 'FA17AntalTrappuppgangar', 'FA20Summa', 'FXByggår','F16AntalPlanUnderMark' , 'FA11Byggnadstyp']]

# bt = bt[bt['FA17AntalTrappuppgangar']<10]
bt
le = LabelEncoder().fit(bt.iloc[:,-1])
# bt.iloc[:,-1] = le.transform(bt.iloc[:,-1])
bt

### XGBoost Classification

In [ ]:
btx = bt.drop(columns=['FXByggår', 'F14ATempt','F16AntalPlanUnderMark'])
btx = btx[(btx['FA11Byggnadstyp'] != 'AnnanVilken') & (btx['FA17AntalTrappuppgangar']<10)] 


# btx.loc[:, 'FA17AntalTrappuppgangar'] = btx.loc[:, 'FA17AntalTrappuppgangar'].fillna(0)
# btx.loc[:, 'FA20Summa'] = btx.loc[:, 'FA20Summa'].fillna(0)
# btx.loc[:, 'FA16AntalPlanOvanMark'] = btx.loc[:, 'FA16AntalPlanOvanMark'].fillna(0)
# btx.loc[:, 'F16AntalPlanUnderMark'] = btx.loc[:, 'F16AntalPlanUnderMark'].fillna(0)

# Separate feature and label
x = btx.iloc[:,0:-1]
y = btx.iloc[:, -1]

# normalization
# x = preprocessing.normalize(x)

# Separate training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.25)

# Classifier
clf = XGBClassifier()
# clf = tree.DecisionTreeClassifier(criterion="entropy")
# clf = RandomForestClassifier()
clf = clf.fit(x_train, y_train)
score = clf.score(x_test, y_test)


# cross validation
# cross_val_score(XGBClassifier(), x, y)
cross_val_score(clf, x, y)

# clf.predict(x_test)
# joblib.dump(clf, 'Buidling_Types_Pre.pkl')

In [ ]:
# type_name = le.inverse_transform(bt.iloc[:,-1])


# plot decision tree
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
import graphviz

# plot single tree
plot_tree(clf)
# plt.show()
plt.savefig('Trees for building type.',dpi=1200)

### Nurual network

In [ ]:
x = bt.iloc[:,0:-1]
y = bt.iloc[:, -1]

sc = StandardScaler()

btn = bt
btn = btn.drop(columns=['FXByggår'])
btn.loc[:, 'FA17AntalTrappuppgangar'] = btn.loc[:, 'FA17AntalTrappuppgangar'].fillna(0)
btn.loc[:, 'FA20Summa'] = btn.loc[:, 'FA20Summa'].fillna(0)
btn.loc[:, 'FA16AntalPlanOvanMark'] = btn.loc[:, 'FA16AntalPlanOvanMark'].fillna(0)
btn.iloc[:, 0:-1] = sc.fit_transform(btn.iloc[:, 0:-1])

# Separate feature and label
x = btn.iloc[:,0:-1]
y = btn.iloc[:, -1]

# Separate training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state = 21)

def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

In [ ]:
#Importing MLPClassifier
from sklearn.neural_network import MLPClassifier

#Initializing the MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(150, 100), 
                           max_iter=600,
                           activation = 'tanh',
                           solver='adam',
                           random_state=2, 
                           alpha=0.01)

#Fitting the training data to the network
classifier.fit(x_train, y_train)


#Predicting y for x_test
y_pred = classifier.predict(x_test)

#Importing Confusion Matrix
from sklearn.metrics import confusion_matrix

#Comparing the predictions against the actual observations in y_val
# cm = confusion_matrix(y_pred, y_test)

cross_val_score(classifier, x, y)

#Printing the accuracy
# print("Accuracy of MLPClassifier: ", accuracy(cm))